In [1]:
# default_exp core
%load_ext autoreload
%autoreload 2

# Aifont Core

> Core functions and constants for Aifont. These are imported by the other Aifont modules.

In [2]:
#export
from fastai.data.all import *
from fastai.vision.all import *
from functools import cmp_to_key
from IPython.display import clear_output
import itertools
from math import sqrt, ceil
from nbdev.showdoc import *
import numpy as np
import pandas as pd
from pandas import DataFrame
from pathlib import Path
import pickle as pkl
from PIL import Image, ImageDraw, ImageFont
import PIL.ImageOps
import skimage.io
import subprocess
import torch
from torch import Tensor
from typing import Any, Callable, List, Protocol, Tuple, Union
from warnings import warn

## Constants and Types

In [3]:
#export
SYS_FONT_PATH = Path("data/system_fonts")
VOCAB_UC = list(string.ascii_uppercase)
VOCAB_UC_NUM = VOCAB_UC + list(string.digits)

ImageTypes         = (PILImage, PILImageBW, Image.Image)
TensorImageOrImage = (TensorImage, *ImageTypes)
TensorOrImage      = (Tensor, *ImageTypes)

# terminal output colours
RED = '\033[91m'
GRN = '\033[92m'
YEL = '\033[93m'
BLU = '\033[94m'
PNK = '\033[95m'
TEL = '\033[96m'
GRY = '\033[97m'
END = '\x1b[0m'

In [4]:
print(f"This is {GRN}green{END} text")

This is green text


## Functions

In [5]:
#export

def clear() -> None:
    """Clear output"""
    clear_output(wait=True)

def report(
    msg: str = "", 
    prog: float = None, 
    line_len = 120
) -> None:
    """Report a one line message with optional progress [0, 1]."""
    if prog is not None: msg = f"{prog:6.2%} • {msg}"
    print(f"{msg:{line_len}s}", end="\r")

def notify(title = "", text = "") -> None:
    """Display an OSX notification."""
    NOTIFY_CMD = '''
    on run argv
        display notification (item 2 of argv) with title (item 1 of argv)
    end run
    '''
    subprocess.call(['osascript', '-e', NOTIFY_CMD, title, text])

def dump(data: Any, filename: Union[Path, str]) -> None:
    """Pickle dump `data`."""
    assert type(filename) is str or isinstance(filename, Path)
    filename = str(filename)
    if not filename.endswith(".pkl"): filename = filename + ".pkl"
    ensure_path(filename)
    with open(filename, 'wb') as f: pkl.dump(data, f, protocol=pkl.HIGHEST_PROTOCOL)

def ensure_path(path: Union[Path, str]) -> Union[Path, str]:
    """Create all folders on `path` if they don't exist. If `path` doesn't have a suffix,
       the last item is created as a folder as well."""
    p = Path(path)
    if p.suffix != "": p = p.parent
    p.mkdir(parents=True, exist_ok=True)
    return path

def load(filename: Union[str, Path]
) -> Any:
    """Pickle load data."""
    filename = Path(filename)
    with open(filename.with_suffix(".pkl"), 'rb') as f: return pkl.load(f)

def try_load(filename: Union[str, Path]
) -> Any:
    """Pickle load data and return `None` if not successful."""
    try: res = load(filename)
    except: return None
    return res

def flatten_list(deep_list: list[list[object]]
) -> list[object]: 
    """Flatten a list of lists."""
    flat = list(itertools.chain.from_iterable(deep_list))
    return flatten_list(flat) if len(flat) and type(flat[0]) is list else flat

def render_text(font_path: Union[Path, str],
    text: str, 
    text_size: int = None, 
    x: int = None, 
    y: int = None, 
    image_width: int = 256, 
    image_height: int = None, 
    as_normalised_array = False
) -> Union[np.ndarray, Image.Image]:
    """Render `text` with `font_path` as black on white and return either 
       as a normalised numpy array of (alpha) values or a PIL Image."""
    assert image_width is not None
    if image_height is None: image_height = image_width
    if text_size is None: text_size = round(image_height * .8)
    if x is None: x = image_width * .5
    if y is None:
        # Try to find a nice y location for the text keeping in mind that descenders
        # will reach below this point and might thus fall off canvas
        y = text_size + max(0, (image_height - text_size * 1.5) / 2)
    font = ImageFont.truetype(str(font_path), text_size)
    # get text info (not being used but may be useful)
    # text_width, text_height = font.getsize(text)
    # left, top, right, bottom = font.getbbox(text)
    # create a blank canvas
    canvas = Image.new('L', (image_width, image_height), 'white')
    # draw the text onto the text canvas
    draw = ImageDraw.Draw(canvas)
    draw.fontmode = "L"
    draw.text((x, y), text, 'black', font, anchor='ms')
    # Convert to normalised list if needed
    return img_to_normalised_array(canvas) if as_normalised_array else canvas

def is_image(x: Tensor) -> bool: return x.ndim >= 2

def is_pil(img) -> bool: return isinstance(img, Image.Image)

def tensor_is_bw_image(x: Tensor, allow_batch = False) -> bool:
    """Check if `x` looks like a black-and-white image. Note that if `allow_batch` is
       `True` a single RGB image will look like a batch of bw images."""
    return (x.ndim == 3 or (x.ndim >  3 and x.shape[-3] == 1)) if allow_batch else \
           (x.ndim == 2 or (x.ndim == 3 and x.shape[-3] == 1))

def tensor_is_rgb_image(x: Tensor, allow_batch = False) -> bool:
    "Check if `x` looks like an RGB image."
    return (x.ndim == 3 or (allow_batch and x.ndim > 3)) and x.shape[-3] == 3

def tensor_to_rgb(x: Tensor, div_int_by = 255.) -> Tensor:
    """Expand `x` to 3 channels and div by `div_int_by` if int. `x` can have 2 to 4 dims.
       The channel dim is expected to be -3 with the preceding ones batches.
       Flattens rgba to rgb."""
    assert x.ndim in (2,3,4)
    if x.ndim == 2:      x = x.expand((3,-1,-1)) # expand 2 to 3 dims
    if x.shape[-1] <= 4: x = x.permute((2,0,1)     if x.ndim == 3 else (0,3,1,2)) # reorder channels first
    if x.shape[-3] == 1: x = x.expand((3,-1,-1))   if x.ndim == 3 else x.expand((-1,3,-1,-1)) # expand grayscale to rgb
    if x.shape[-3] == 4: x = x[0:3,:,:] * x[3,:,:] if x.ndim == 3 else x[:,0:3,:,:] * x[:,3,:,:] # flatten rgba
    assert x.shape[-3] == 3
    return x if torch.is_floating_point(x) else x / div_int_by

def img_from_tensor(t: Tensor, 
    scale: float = None
) -> Image.Image:
    """Create an image from a tensor produced by `aifont.fontlearner.VectorRenderLayer`."""
    pil_img = to_image(tensor_to_rgb(t.clip(0., 1.).squeeze()))
    if scale is not None: pil_img = pil_img.resize((round(pil_img.width * scale), round(pil_img.height * scale)), resample=0)
    return pil_img

def img_to_normalised_array(img: Image.Image) -> np.ndarray:
    """Convert a rendered image to a normalised array."""
    return np.reshape([x/255. for x in list(img.getdata())], (img.height, img.width))

def image_grid(imgs: Union[List[Image.Image], List[str], Tensor], 
    rows: int = None, 
    cols: int = None,
    bg_color: any = "white",
    scale: float = None,
) -> Image.Image:
    """Create a grid from the images or image filenames supplied."""
    if isinstance(imgs, Tensor):
        if   imgs.ndim == 2: imgs = imgs.reshape(( 1,1,*imgs.shape[-2:]))
        elif imgs.ndim == 3: imgs = imgs.reshape((-1,*imgs.shape[-3:]))
        elif imgs.ndim != 4: raise ValueError(f"Can't deal with tensor of {imgs.ndim} dimensions.")
        imgs = [img_from_tensor(o.detach()) for o in imgs]
    elif type(imgs[0]) is str: imgs = [Image.open(o) for o in imgs]
    n = len(imgs)
    if rows is None: rows = ceil(sqrt(n)) if cols is None else ceil(n / cols)
    if cols is None: cols = ceil(n / rows)
    w, h = imgs[0].size
    size = (cols * w, rows * h)
    grid = Image.new('RGB', size=size, color=bg_color)
    for i, img in enumerate(imgs): grid.paste(img, box=(i % cols * w, i // cols * h))
    if scale is not None: grid = grid.resize((round(o * scale) for o in size))
    return grid

def df_row_to_image(
    df_row: pd.Series,
    start_col = 1
) -> Image:
    """Convert image data from a DataFrame row to an Image object."""
    sz = len(df_row) - start_col
    wh = int(sz ** .5)
    assert sz == wh ** 2
    img = df_row[start_col : len(df_row)].values.reshape(wh, wh)
    return Image.fromarray(img)

@delegates(image_grid)
def df_to_image_grid(
    df: DataFrame,
    start_col = 1,
    max_images = 50,
    invert = False,
    **kwargs,
) -> Image:
    """Convert a DataFrame to an Image grid and optionally `invert`."""
    imgs = [df_row_to_image(df.iloc[i], start_col=start_col) for i in range(min(len(df), max_images))]
    if invert: imgs = [PIL.ImageOps.invert(o) for o in imgs]
    return image_grid(imgs, **kwargs)

def read_image(fn: str) -> Tensor:
    """Read an image and return a normalised rgba tensor."""
    return Tensor(skimage.io.imread(fn) / 255.)

def s(x):
    if not isinstance(x, Tensor): x = torch.tensor(x)
    return 1/(1 + torch.exp(-x))

def smooth(y, 
    dist = 1
) -> list:
    """Smooth values in the list up to `dist` items in both directions."""
    assert dist > 0
    if type(y) is pd.Series: y = y.to_numpy()
    window = 2 * dist + 1
    y = np.insert(y, 0, [0] + [y[0]] * dist)
    y = np.append(y, [y[-1]] * dist)
    cs = np.cumsum(y)
    return (cs[window:] - cs[:-window]) / window

def plot_function(f, 
    tx=None, ty=None, 
    title=None, 
    min=-2, max=2, 
    figsize=(6,4)
) -> None:
    """From fastbook.utils"""
    import matplotlib as mpl
    mpl.rcParams['savefig.dpi']= 200
    mpl.rcParams['font.size']=12
    x = torch.linspace(min,max,steps=None)
    fig,ax = plt.subplots(figsize=figsize)
    ax.plot(x,f(x))
    if tx is not None: ax.set_xlabel(tx)
    if ty is not None: ax.set_ylabel(ty)
    if title is not None: ax.set_title(title)

def concat_tensors(*tensors,
    flatten = False
) -> Tensor:
    """Utility to concat tensors or numbers even if they are zero-dimensional.
       Optionally `flatten` the concatenated tensor."""
    if type(tensors[0]) in (list, tuple): 
        assert len(tensors) == 1
        tensors = tensors[0]
    tt = [tensor(x) for x in tensors]
    res = torch.concat([x.unsqueeze(0) if x.ndim == 0 else x for x in tt])
    return res.flatten() if flatten else res

def file_cmp(a, b, sep=r"[\W_\-]+") -> int:
    "Compare to strings by splitting them and comparing each part with numbers compared as such."
    for a_,b_ in zip(*[re.split(sep, o) for o in (a,b)]):
        if a_.isdigit(): a_,b_ = int(a_),int(b_)
        if   a_ < b_: return - 1
        elif a_ > b_: return 1
    return 0

def get_file_sorter(sep=None) -> Callable:
    """Get a `key` function for `sort` to compare to strings by splitting them and 
       comparing each part with numbers compared as such."""
    return cmp_to_key(file_cmp if sep is None else partial(file_cmp, sep=sep))

def mean(vals: list) -> float:
    "Get the mean of `vals`."
    return np.array(vals).mean()

## Export

In [5]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 01_experiment_1_match_font.ipynb.
Converted 02_experiment_2_ocr_models.ipynb.
Converted 03_experiment_3_optimise_with_ocr.ipynb.
Converted 04_font_statistics.ipynb.
Converted aifont_core.ipynb.
Converted aifont_fontlearner.ipynb.
Converted aifont_fontsampler.ipynb.
Converted aifont_ocrlearner.ipynb.
Converted index.ipynb.
